<a href="https://colab.research.google.com/github/kuds/rl-drone/blob/main/RL%20Drone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install mujoco
!pip install robot_descriptions
!pip install gymnasium
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.8/99.8 kB 3.3 MB/s eta 0:00:00


In [4]:
import gymnasium as gym
from gymnasium import spaces
import mujoco
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

drone_xml = """
<mujoco model="drone">
  <compiler angle="degree" coordinate="local" inertiafromgeom="true"/>
  <option integrator="RK4" timestep="0.01"/>
  <worldbody>
    <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
    <body name="drone" pos="0 0 1">
      <camera name="track" mode="trackcom" pos="0 -3 1" xyaxes="1 0 0 0 0 1"/>
      <geom type="box" size=".1 .1 .02" rgba=".9 .9 .9 1" mass="1"/>
      <joint type="free" limited="false"/>
      <!-- Rotors -->
      <body name="rotor1_body" pos="0.1 0.1 0">
        <joint name="rotor1" type="hinge" axis="0 0 1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="0 1 0 1"/>
      </body>
      <body name="rotor2_body" pos="-0.1 0.1 0">
        <joint name="rotor2" type="hinge" axis="0 0 -1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="1 0 0 1"/>
      </body>
      <body name="rotor3_body" pos="0.1 -0.1 0">
        <joint name="rotor3" type="hinge" axis="0 0 -1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="1 0 0 1"/>
      </body>
      <body name="rotor4_body" pos="-0.1 -0.1 0">
        <joint name="rotor4" type="hinge" axis="0 0 1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="0 1 0 1"/>
      </body>
    </body>
    <!-- Target -->
    <body name="target" pos="1 1 2">
        <geom type="sphere" size="0.1" rgba="1 0 0 0.5" />
    </body>
  </worldbody>
  <actuator>
    <motor joint="rotor1" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
    <motor joint="rotor2" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
    <motor joint="rotor3" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
    <motor joint="rotor4" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
  </actuator>
</mujoco>
"""

class DroneEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self):
        super(DroneEnv, self).__init__()

        # Load the drone model
        self.model = mujoco.MjModel.from_xml_string(drone_xml)
        self.data = mujoco.MjData(self.model)

        # Define action and observation space
        # Actions are the 4 rotor thrusts
        self.action_space = spaces.Box(low=0, high=1, shape=(4,), dtype=np.float32)

        # Observations: drone_pos(3), drone_vel(3), drone_orient(4), target_pos(3)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(13,), dtype=np.float32)

        # Target position
        self.target_position = np.array([1.0, 1.0, 2.0])

    def step(self, action):
        # Apply action
        self.data.ctrl[:] = action * 1000  # Scale action to actuator range
        mujoco.mj_step(self.model, self.data)

        # Get observation
        obs = self._get_obs()

        # Calculate reward
        drone_pos = self.data.qpos[0:3]
        dist_to_target = np.linalg.norm(drone_pos - self.target_position)

        # Reward for being close to the target
        reward = -dist_to_target

        # Penalty for being tilted
        orientation = self.data.qpos[3:7]
        # A simple measure of tilt: z-component of the drone's up vector
        # For a quaternion [w, x, y, z], the z-axis of the rotated frame is
        # [2*(xz + wy), 2*(yz - wx), 1 - 2*(x^2 + y^2)]
        up_z = 1 - 2 * (orientation[1]**2 + orientation[2]**2)
        reward -= (1 - up_z) * 0.5

        # Penalty for control effort
        reward -= np.sum(action) * 0.01

        # Check if done
        done = False
        if dist_to_target < 0.2:
            reward += 100  # Bonus for reaching the target
            done = True
        if drone_pos[2] < 0.1: # Crashed
            reward -= 100
            done = True

        return obs, reward, done, False, {}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        # Reset the simulation
        mujoco.mj_resetData(self.model, self.data)

        # Randomize target position
        self.target_position = np.random.rand(3) * 2 + np.array([-1, -1, 1])
        self.model.body('target').pos[:] = self.target_position

        return self._get_obs(), {}

    def _get_obs(self):
        return np.concatenate([
            self.data.qpos[0:3],    # Drone position
            self.data.qvel[0:3],    # Drone linear velocity
            self.data.qpos[3:7],    # Drone orientation (quaternion)
            self.target_position
        ])

    def render(self, mode='human'):
        # This is a placeholder for rendering.
        # For a full visualization, you would need to integrate a renderer
        # like mujoco.viewer.
        pass

# Create a vectorized environment
env = DummyVecEnv([lambda: DroneEnv()])

# Instantiate the PPO model
model = PPO("MlpPolicy", env, verbose=1)

# Train the model
training_timesteps = 10000
model.learn(total_timesteps=training_timesteps)

# Save the model
model.save("ppo_drone")

print("\nTraining finished and model saved as 'ppo_drone.zip'")

# Load the trained model
model = PPO.load("ppo_drone")

# Create a new environment for evaluation
eval_env = DroneEnv()
obs, _ = eval_env.reset()

print("\nStarting evaluation...")
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, _, info = eval_env.step(action)
    if dones:
        print(f"Episode finished after {i+1} timesteps.")
        obs, _ = eval_env.reset()

print("Evaluation finished.")

Using cpu device
-----------------------------
| time/              |      |
|    fps             | 1186 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 732         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007251151 |
|    clip_fraction        | 0.0655      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.67       |
|    explained_variance   | 0.0105      |
|    learning_rate        | 0.0003      |
|    loss                 | 1.07e+03    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0103     |
|    std                  | 0.996       |
|    value_loss           | 2.69e+03    |
-----------------

In [13]:
import mujoco

# Loading a specific model description as an imported module.
from robot_descriptions import cf2_mj_description
model = mujoco.MjModel.from_xml_path(cf2_mj_description.MJCF_PATH)

In [21]:
import mujoco
import robot_descriptions.cf2_mj_description as cf2_mj_description
import xml.etree.ElementTree as ET

# Read the XML content from the file path
tree = ET.parse(cf2_mj_description.MJCF_PATH)
root = tree.getroot()

# Find the worldbody element
worldbody = root.find('worldbody')

# Define the sphere element as a string
sphere_xml = """
<body name="sphere_body" pos="0 0 0.5">
    <geom type="sphere" size="0.1" rgba="0 0 1 1"/>
</body>
"""

# Parse the sphere XML string
sphere_element = ET.fromstring(sphere_xml)

# Append the sphere element to the worldbody
worldbody.append(sphere_element)

# Convert the modified XML back to a string
modified_xml_string = ET.tostring(root, encoding='unicode')

# Load the modified model, providing the asset directory
# Note: robot_descriptions.cf2_mj_description does not have ASSETS_PATH.
# You would need to find the correct way to access the asset path for this model.
# For now, I will load without asset_dir, which might cause issues if the model
# relies on external assets.
modified_model = mujoco.MjModel.from_xml_string(modified_xml_string)

# You can now use modified_model for simulation or visualization
print("Sphere added to the model.")

# Example: create data for the modified model
data = mujoco.MjData(modified_model)

# Example: print body names to verify the sphere body is added
for i in range(modified_model.nbody):
    print(f"Body ID: {i}, Name: {modified_model.body(i).name}")

ValueError: Error: Error opening file 'assets/cf2_0.obj': No such file or directory